In [1]:
import os
import pickle
import holidays
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

**Temperature preprocessing**

In [2]:
all_temp_df = pd.read_excel("data/raw/Daily Regional Temperature.xlsx")  
all_temp_df.head()

,Date,Code INSEE région,Région,TMin (°C),TMax (°C),TMoy (°C)
0,2016-01-01,24,Centre-Val de Loire,1.52,10.73,6.12
1,2016-01-01,11,Île-de-France,3.30,8.76,6.03
2,2016-01-01,75,Nouvelle-Aquitaine,3.32,13.58,8.45
3,2016-01-01,93,Provence-Alpes-Côte d'Azur,5.30,12.77,9.03
4,2016-01-01,27,Bourgogne-Franche-Comté,2.22,9.52,5.87


In [3]:
temp_df = all_temp_df[all_temp_df['Date'].between('2016-01-01','2019-12-31')]
temp_df.tail()

,Date,Code INSEE région,Région,TMin (°C),TMax (°C),TMoy (°C)
18988,2019-12-31,27,Bourgogne-Franche-Comté,-2.90,3.03,0.07
18989,2019-12-31,44,Grand Est,-4.03,5.07,0.52
18990,2019-12-31,28,Normandie,-1.98,4.60,1.31
18991,2019-12-31,84,Auvergne-Rhône-Alpes,-2.77,7.66,2.45
18992,2019-12-31,94,Corse,2.50,13.70,8.10


In [4]:
temp_df.rename(columns={'Code INSEE région':'Code'}, inplace=True)

``` remove all rows belonging to coarse island region coded by 94```

In [5]:
temp_df.drop(temp_df.index[temp_df["Code"] == 94],inplace=True)
temp_df.head()

,Date,Code,Région,TMin (°C),TMax (°C),TMoy (°C)
0,2016-01-01,24,Centre-Val de Loire,1.52,10.73,6.12
1,2016-01-01,11,Île-de-France,3.30,8.76,6.03
2,2016-01-01,75,Nouvelle-Aquitaine,3.32,13.58,8.45
3,2016-01-01,93,Provence-Alpes-Côte d'Azur,5.30,12.77,9.03
4,2016-01-01,27,Bourgogne-Franche-Comté,2.22,9.52,5.87


```Check if there is an invalid date```

In [6]:
bool_col=pd.to_datetime(temp_df.Date, format='%Y-%m-%d', errors='coerce').isna()
invalid_date=temp_df.Date[bool_col]
print(invalid_date)

Series([], Name: Date, dtype: datetime64[ns])


```Check whether if there are some cells of the dataframe containing some NA value```

In [7]:
temp_df.isnull().sum()

Date         0
Code         0
Région       0
TMin (°C)    0
TMax (°C)    0
TMoy (°C)    0
dtype: int64

```Convert the date column as an index to helps us to access and analyze the data easier without any condition```

In [8]:
temp_df.set_index('Date',inplace=True)
temp_df.loc["2016-06"].head()

,Code,Région,TMin (°C),TMax (°C),TMoy (°C)
Date,,,,,
2016-06-01,24,Centre-Val de Loire,11.83,17.53,14.68
2016-06-01,32,Hauts-de-France,12.62,14.92,13.77
2016-06-01,93,Provence-Alpes-Côte d'Azur,12.98,20.50,16.74
2016-06-01,11,Île-de-France,13.34,17.00,15.17
2016-06-01,28,Normandie,11.78,14.16,12.97


```Check if all regions have the same number of observations.```

In [9]:
temp_df.groupby(['Code']).size()

Code
11    1461
24    1461
27    1461
28    1461
32    1461
44    1461
52    1461
53    1461
75    1461
76    1461
84    1461
93    1461
dtype: int64

```Check if there is a missing date in the dataset```

In [10]:
missing_date = pd.date_range(start = temp_df.index.min(), end = temp_df.index.max()
                            ).difference(temp_df.index).strftime('%Y-%m-%d').tolist()
print(missing_date)

[]


```Add the quarter column```

In [11]:
temp_df.insert(loc=temp_df.columns.shape[0], column='Quarter', value=temp_df.index.quarter)

```Add workday column, workday = 1, weekend = 0```

In [12]:
temp_df["Weekday"] = temp_df.index.weekday 

```Add holilday column ```

In [13]:
temp_df["Holiday"] = 0
holdidays = temp_df["Weekday"] >= 5
temp_df.loc[holdidays,"Holiday"] = 1

public_holidays = temp_df.index.to_series().apply(lambda x:x in holidays.France())
public_holidays_dates= temp_df.index[public_holidays]
temp_df.loc[public_holidays_dates,"Holiday"] = 1

In [14]:
non_work_days = temp_df[temp_df["Holiday"] == 1]["Holiday"].count()/12
work_days = temp_df[temp_df["Holiday"] == 0]["Holiday"].count()/12
print("Number of weekends and public holidays per region and years (2016-2019): {}".format(non_work_days))
print("Number of workdays per region and years (2016-2019): {}".format(work_days))

Number of weekends and public holidays per region and years (2016-2019): 454.0
Number of workdays per region and years (2016-2019): 1007.0


```Add HDD and CDD column based on https://en.wikipedia.org/wiki/Heating_degree_day#cite_note-1```

In [15]:
# 59.9 F = 15.5 °C
temp_df["HDD"] = np.where( temp_df["TMoy (°C)"] < 15.5, 15.5 - temp_df["TMoy (°C)"], 0)
temp_df["CDD"] = np.where( temp_df["TMoy (°C)"] > 15.5, temp_df["TMoy (°C)"] - 15.5, 0)

In [16]:
temp_dict = {}
region = temp_df.groupby(["Code"])
for name,group in region:
    temp_dict[name] = group

In [17]:
os.makedirs('data/processed', exist_ok=True)
with open('data/processed/temp_data.pkl', 'wb') as f:
    pickle.dump(temp_dict, f)